# Definition of the different models 
### Notation
$n_1, n_2$: cytokine concentrations projected in latent space

$ N_1, N_2$: cytokine integrals projected in latent space

## Constant velocity model
See supplementary information

## Constant force model
Intermediate model between constant velocity and force model with matching; removed from the paper. 

## Force model with matching ("Sigmoid")
There are two versions: "Sigmoid_freealpha", in which kinetic rate paraemters $\alpha$ and $\beta$ are fitted separately, and "Sigmoid_fixalpha", in which $\alpha = \frac{1}{20} \,  \mathrm{h^{-1}}$ and $\gamma = \beta / \alpha$ is fitted. 

### Equations for $n_1$, $n_2$
Similar for both nodes, but to capture the initial dip in node 2, we need to square the bounded exponential of the first phase

$$ n_1(t) = \left(1 - e^{-\alpha t} \right) \left( \frac{a_0 \cos{\theta} + v_1}{e^{\beta(t - t_0)} + 1} - v_1 \right)$$

$$ n_2(t) = \left(1 - e^{-\alpha t} \right) \left( \frac{(a_0 \sin{\theta} + v_2)(1 - e^{-\alpha t})}{e^{\beta(t - t_0)} + 1} - v_2 \right)$$

### Equations for $N_1$, $N_2$
It is possible to integrate analytically, so we can fit integrals first. Define $\tau = \alpha t$, $\tau_0 = \alpha t_0$, and $\gamma = \beta / \alpha$. The result is


$$ N_1(t) = \frac{a_0 \cos{\theta}+ v_1}{\alpha} \left( I(\tau, \tau_0, \gamma) - \frac{1}{\gamma} \ln{\left(e^{-\gamma \tau} + e^{-\gamma \tau_0}  \right)} \right)  
- \frac{v_1}{\alpha} \left(\tau + e^{-\tau} \right) + K_1 $$

$$ N_2(t) = \frac{a_0 \sin{\theta} + v_2}{\alpha} \left( 2I(\tau, \tau_0, \gamma) - \tfrac12 I(2 \tau, 2\tau_0, \tfrac{\gamma}{2}) - \tfrac{1}{\gamma} \ln{\left(e^{-\gamma \tau} + e^{-\gamma \tau_0}  \right)} \right)  
- \frac{v_2}{\alpha} \left(\tau + e^{-\tau} \right) + K_2 $$ 

where the $K_i$ are chosen to ensure $N_i(0) = 0$. The complicated part is the integral $I(\tau, \tau_0, \gamma)$, which is

$$ I(\tau, \tau_0, \gamma) = \int \mathrm{d} \tau \frac{-e^{-\tau}}{e^{\gamma (\tau - \tau_0) }+ 1} $$

The special case $\frac{1}{\gamma} \in \mathbb{N}^+$ can be solved using partial fractions:


$$ I(\tau) = e^{-\tau} + e^{-\tau} \sum_{j=1}^{n-1} \frac{n}{n-j} (-1)^j e^{(\tau - \tau_0)\frac{j}{n}} \\
+ (-1)^n n e^{-\tau_0} \ln{ \left(e^{-\tau/n} + e^{-\tau_0 / n} \right)} $$

For the general case, Mathematica suggests the following answer, valid  when $\frac{1}{\gamma} \notin \mathbb{N}^+$:

$$ I(\tau) = e^{-\tau} {}_2F_1(1, \frac{-1}{\gamma}; 1 - \frac{1}{\gamma}; -e^{\gamma(\tau - \tau_0)}) $$

where ${}_2F_1$ is Gauss' hypergeometric function. This result can be derived from ${}_2F_1(0, b; c; z) =1$ and from the recurrence relation 
(NIST's *Digital Library of Mathematical Formulas*, eq. 15.5.20)

$$ z(1-z) \frac{d}{dz} {}_2F_1(a, b; c; z) = (c-a) \, {}_2F_1(a-1, b; c; z) \\ + (a -c +bz) \, {}_2F_1(a, b; c; z) $$

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import sys, os
from time import perf_counter  # For timing
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [2]:
from ltspcyt.scripts.adapt_dataframes import set_standard_order, sort_SI_column
from ltspcyt.scripts.latent_space import import_mutant_output
from ltspcyt.scripts.neural_network import import_WT_output

# Curve fitting functions
from ltspcyt.scripts.sigmoid_ballistic import return_param_and_fitted_latentspace_dfs

In [3]:
from scipy.special import seterr as special_seterr
special_seterr(loss="warn");

In [4]:
%matplotlib inline

In [5]:
peptides=["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3", "A8", "Q7"]
concentrations=["1uM","100nM","10nM","1nM"]
fit_vars={"Constant velocity":["v0","t0","theta","vt"],"Constant force":["F","t0","theta","vt"],
         "Sigmoid":["a0", "t0", "theta", "v1", "gamma"], 
         "Sigmoid_freealpha":["a0", "t0", "theta", "v1", "alpha", "beta"]}

In [6]:
df_WT=import_WT_output()
df_min,df_max=pd.read_pickle(os.path.join("data", "trained-networks", "min_max-thomasRecommendedTraining.pkl"))
projmat = np.load(os.path.join("data", "trained-networks", "mlp_input_weights-thomasRecommendedTraining.npy"))
print(df_min)

PeptideComparison_OT1_Timeseries_19.hdf
HighMI_5-m1r4.hdf
DrugPerturbation_4.hdf
PeptideComparison_OT1_Timeseries_18.hdf
DrugPerturbation_6.hdf
PeptideComparison_OT1_Timeseries_23.hdf
DrugPerturbation_3.hdf
HighMI_5-m1r2.hdf
CAR_Transduction_OT1_Timeseries_1.hdf
HighMI_5-m1r3.hdf
DrugPerturbation_2.hdf
PeptideComparison_OT1_Timeseries_22.hdf
PeptideComparison_OT1_Timeseries_9.hdf
PeptideComparison_OT1_Timeseries_20.hdf
HighMI_5-m1r1.hdf
ITK--_OT1_Timeseries_2.hdf
PeptideComparison_OT1_Timeseries_21.hdf
HighMI_1-4.hdf
LilyDrugPerturbation_2.hdf
TCellTypeComparison_OT1,P14,F5_Timeseries_2.hdf
HighMI_5-m2r4.hdf
HighMI_13-1.hdf
TCellNumber_OT1_Timeseries_7.hdf
HighMI_13-3.hdf
Activation_Timeseries_1.hdf
ITAMDeficient_OT1_Timeseries_8.hdf
ITAMDeficient_OT1_Timeseries_9.hdf
TCellTypeComparison_OT1,P14,F5_Timeseries_1.hdf
HighMI_13-2.hdf
OT1_CAR_CytokineOnly_2.hdf
AntagonismComparison_OT1,P14,F5_Timeseries_2.hdf
PeptideTumorComparison_OT1_Timeseries_2.hdf
HighMI_5-m2r3.hdf
HighMI_1-2.hdf
High

In [7]:
cytokines=df_min.index.get_level_values("Cytokine")
times=np.arange(1,73)

In [8]:
df=df_WT.unstack("Time").loc[:,("integral",cytokines,times)].stack("Time")
df=(df - df_min)/(df_max - df_min)
df_proj=pd.DataFrame(np.dot(df, projmat),index=df.index,columns=["Node 1","Node 2"])

In [9]:
# Defining the dataset(s) to use
# Multiple datasets to populate parameter space better
subset_exp = [
    "Activation_2",
    "Activation_TCellNumber_1",
    "Activation_Timeseries_1",
    "CD25MutantTimeSeries_OT1_Timeseries_2",
    "NewPeptideComparison_OT1_Timeseries_20",
    "PeptideComparison_OT1_Timeseries_18",
    "PeptideComparison_OT1_Timeseries_19",
    "PeptideComparison_OT1_Timeseries_20",
    "TCellNumber_1",
    "TCellNumber_2",
    "TCellNumber_OT1_Timeseries_7", 
    "HighMI_1-1", "HighMI_1-2", "HighMI_1-3", "HighMI_1-4"
]
df_proj_exp = df_proj.loc[subset_exp]

# Or with a single data set to go faster
#subset_exp = ["NewPeptideComparison_OT1_Timeseries_20"]
#df_proj_exp=df_proj.loc[(subset_exp, slice(None)), :]

In [17]:
# Model choice and curve fit of that model
fit="Sigmoid_freealpha"
regul_rate = 0.4  # Use something like 0.4 if "Sigmoid" or "Sigmoid_freealpha", 1.0 if "Constant velocity"

# File names specification for that model and regularization rate
name_specs = "{}_reg{}".format(fit.replace(" ", "_"), str(round(regul_rate, 2)).replace(".", ""))

start_time = perf_counter()

ret = return_param_and_fitted_latentspace_dfs(df_proj_exp, fit, reg_rate=regul_rate)
df_params, df_compare, df_hess, df_v2v1 = ret

end_t = perf_counter()
print("Time to fit: ", perf_counter() - start_time)
del start_time

nparameters = len(fit_vars[fit])
print(df_hess.median())  # Hessian matrix: inverse of the covariance. 

Activation_2
	Median slope:	 3.87
	Mean slope:	 4.28
Activation_TCellNumber_1
	Median slope:	 3.34
	Mean slope:	 3.69
Activation_Timeseries_1
	Median slope:	 5.88
	Mean slope:	 5.08
CD25MutantTimeSeries_OT1_Timeseries_2
	Median slope:	 6.3
	Mean slope:	 8.67
NewPeptideComparison_OT1_Timeseries_20
	Median slope:	 7.48
	Mean slope:	 3.72
PeptideComparison_OT1_Timeseries_18
	Median slope:	 7.39
	Mean slope:	 6.2
PeptideComparison_OT1_Timeseries_19
	Median slope:	 10.57
	Mean slope:	 9.61
PeptideComparison_OT1_Timeseries_20
	Median slope:	 7.48
	Mean slope:	 3.72
TCellNumber_1
	Median slope:	 5.27
	Mean slope:	 5.72
TCellNumber_2
	Median slope:	 2.14
	Mean slope:	 2.07
TCellNumber_OT1_Timeseries_7
	Median slope:	 3.96
	Mean slope:	 3.44
HighMI_1-1
	Median slope:	 5.95
	Mean slope:	 6.11
HighMI_1-2
	Median slope:	 5.99
	Mean slope:	 -0.12
HighMI_1-3
	Median slope:	 5.89
	Mean slope:	 3.36
HighMI_1-4
	Median slope:	 6.27
	Mean slope:	 6.33
Time to fit:  52.117134663
a0     a0       5.153684e

In [ ]:
dataset = subset_exp[1]  # Select the data set to plot here
tcellnum = "100k"
df_compare_sel = df_compare.xs(tcellnum, level="TCellNumber", axis=0).xs(dataset, level="Data", axis=0)
print(df_compare_sel.index.names)
df_compare_sel.columns.names = ["Node"]
data=df_compare_sel.loc[(peptides,concentrations,slice(None),slice(None),"concentration"),:]
h=sns.relplot(data=data.stack().reset_index(),x="Time",y=0,kind="line",sort=False,
            hue="Peptide",hue_order=peptides,
            col="Concentration",col_order=concentrations,row="Node",
            style="Processing type")
# h.fig.savefig("figures/fits/concentrations_{}_{}.pdf".format(name_specs, dataset))
plt.show()
plt.close()

In [ ]:
# Plotting the fit of integrals vs time
data=df_compare_sel.loc[(peptides,concentrations,slice(None),slice(None),"integral"),:]
h=sns.relplot(data=data.stack().reset_index(),x="Time",y=0,kind="line",sort=False,
            hue="Peptide",hue_order=peptides,
            col="Concentration",col_order=concentrations,row="Node",
            style="Processing type")
# h.fig.savefig("figures/fits/integrals_{}_{}.pdf".format(name_specs, dataset))
plt.show()
plt.close()

In [ ]:
# Plotting the latent space ballistic trajectories N_1 vs N_2
data=df_compare_sel.loc[(peptides,concentrations,slice(None),slice(None),"integral"),:]
h=sns.relplot(data=data.reset_index(), x="Node 1",y="Node 2", kind="line", sort=False,
            hue="Peptide",hue_order=peptides,
            size="Concentration",size_order=concentrations,
            style="Processing type")
# h.fig.savefig("figures/fits/N1_vs_N2_{}_{}.pdf".format(name_specs, dataset), transparent=True)
plt.show()
plt.close()

In [ ]:
h = sns.pairplot(data=df_params.loc[df_params.index.isin(peptides, level="Peptide")].reset_index(), 
                 vars=fit_vars[fit], hue="Peptide", hue_order=peptides)
legend = h.legend

#h.fig.savefig("figures/fits/pairplot_{}_selectdata.pdf".format(name_specs), 
#    transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
tcn_select = ["100k", "30k", "10k", "3k"]
h = sns.pairplot(data=df_params.loc[df_params.index.isin(tcn_select, level="TCellNumber")].reset_index(), 
                 vars=fit_vars[fit], hue="TCellNumber", hue_order=tcn_select, palette="viridis")
legend = h.legend

#h.fig.savefig("figures/fits/pairplot_tcellnumcolor_{}_selectdata.pdf".format(name_specs), 
#    transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
plt.show()
plt.close()

In [18]:
# To save df_compare to be reused elsewhere for plotting, e.g. main_plotting_scripts
df_compare.to_hdf(os.path.join("results", "fits", "df_compare_{}_selectdata.hdf".format(name_specs)), key="df_compare", mode="w")
df_params.to_hdf(os.path.join("results", "fits", "df_params_{}_selectdata.hdf".format(name_specs)), key="df_params", mode="w")